In [ ]:
type Node{T}
    left::Nullable{Node{T}}
    right::Nullable{Node{T}}
    value::T
end

Node{T}(v::T) = Node(Nullable{Node{T}}(), Nullable{Node{T}}(), v)

In [1]:
n1 = Node(Nullable(Node(2)), Nullable(Node(3)), 10)

LoadError: LoadError: UndefVarError: Node not defined
while loading In[1], in expression starting on line 1

In [12]:
left(n::Node) = isnull(n.left) ? Void(): get(n.left)
right(n::Node) = isnull(n.right) ? Void(): get(n.right)

right (generic function with 1 method)

In [1]:
function search{T}(graph::Node{T}, number)
    for i = 1:10
        if graph.value == number
            return number
        else
            try
                graph = graph.left.value
            catch
                return 0
            end
        end
    end
end
    

LoadError: LoadError: UndefVarError: Node not defined
while loading In[1], in expression starting on line 1

In [1]:
include("SymmetricMatrix.jl")
include("Test.jl")
using Test

In [5]:
bs = (convert(BoxStructure{Float64}, st, 10)) ;
@time bs1 = (convert(BoxStructure{Float64}, st1, 10)) ;

  0.368854 seconds (320.15 k allocations: 589.424 MB, 32.72% gc time)


In [6]:
@time bs+bs1;
@time bs+2;
@time st + st1;
@time st + 2;

  0.107971 seconds (259.60 k allocations: 13.942 MB)
  0.080173 seconds (322.12 k allocations: 17.625 MB, 4.54% gc time)
  0.013778 seconds (7 allocations: 47.684 MB, 19.70% gc time)
  0.012109 seconds (7 allocations: 47.684 MB, 10.60% gc time)


In [7]:
@time convert(Array{Float64}, bs);
@time vec(bs);
@time vec(st);

  0.338190 seconds (1.23 M allocations: 148.155 MB, 15.07% gc time)
  0.297563 seconds (1.21 M allocations: 147.300 MB, 15.78% gc time)
  0.000001 seconds (6 allocations: 240 bytes)


In [8]:
@time bs.*bs1;
@time bs+bs1;
@time bs./bs1;
@time st.*st1;
@time st+st1;
@time st./st1;

  0.046923 seconds (217.14 k allocations: 12.077 MB, 5.30% gc time)
  0.041683 seconds (211.89 k allocations: 11.764 MB)
  0.046537 seconds (217.14 k allocations: 12.077 MB, 5.47% gc time)
  0.018239 seconds (13 allocations: 47.684 MB, 7.72% gc time)
  0.015898 seconds (7 allocations: 47.684 MB, 4.35% gc time)
  0.049982 seconds (13 allocations: 47.684 MB, 1.66% gc time)


In [8]:
@time modemult(bs, c', 4)
@time Tensors.modemult(st,c',4);

  3.900043 seconds (5.54 M allocations: 6.035 GB, 6.03% gc time)
  1.293228 seconds (176 allocations: 946.063 MB, 2.76% gc time)


In [ ]:
l = 75
st = genstensor(Float64, 4,l);
st1 = genstensor(Float64, 4,l, 1233);
c = randn(l,12);

In [2]:
T1 = randn(5,5);
r = randn(25, 25)
r1 = randn(25,10)
c = covbs(r, 5);
c1 = cov(r, corrected = false);



In [5]:
t = genstensor(Float64, 3,25)
bs = convert(BoxStructure{Float64}, t, 5);

In [9]:
a = convert(BoxStructure{Float64}, t, 5)
a.frame[1,1,1].value

5x5x5 Array{Float64,3}:
[:, :, 1] =
  0.867347  -0.901744    0.864401  -0.560501    1.56417 
 -0.901744  -0.494479    2.21188   -0.0192918  -1.39674 
  0.864401   2.21188    -0.271735   1.85278    -1.10673 
 -0.560501  -0.0192918   1.85278   -0.251176    0.150976
  1.56417   -1.39674    -1.10673    0.150976   -1.27967 

[:, :, 2] =
 -0.901744   -0.494479   2.21188   -0.0192918  -1.39674 
 -0.494479   -0.902914   0.532813   0.128064    1.1055  
  2.21188     0.532813   0.502334  -0.827763   -3.21136 
 -0.0192918   0.128064  -0.827763   0.369714    0.769278
 -1.39674     1.1055    -3.21136    0.769278    0.997317

[:, :, 3] =
  0.864401   2.21188   -0.271735    1.85278    -1.10673  
  2.21188    0.532813   0.502334   -0.827763   -3.21136  
 -0.271735   0.502334  -0.516984    0.110096   -0.0740145
  1.85278   -0.827763   0.110096    0.0721164  -0.310153 
 -1.10673   -3.21136   -0.0740145  -0.310153    0.302423 

[:, :, 4] =
 -0.560501   -0.0192918   1.85278    -0.251176    0.150976
 -0.01

In [6]:
seg(i::Int, of::Int, limit::Int) = (i*of <= limit)? (((i-1)*of+1):(i*of)): (((i-1)*of+1):limit)
@generated function slise1{T <: AbstractFloat, N}(mat::Array{T, N}, slisesize::Int)
    quote
        segments = ceil(Int, [size(mat)...]/slisesize)
        ret = NullableArray(Array{T, N}, segments...)
        @nloops $N i x -> 1:segments[x] begin
            ind = @ntuple $N x -> i_{$N-x+1}
            ret[ind...] = mat[(map(i -> seg(ind[i], slisesize, size(mat,i)), 1:N))...]
        end
        ret
    end
end


slise1 (generic function with 1 method)

In [4]:
include("SymmetricMatrix.jl")
using SymmetricMatrix
using NullableArrays
importall SymmetricMatrix

using Iterators

function genstensor(T::Type, dims::Int, l::Int, seed::Int = 1234)
    srand(seed)
    ret = zeros(T, fill(l, dims)...)
    indices = collect(product(fill(collect(1:l), dims)...))
    elements = Array{T,1}(randn(l^dims÷dims))
    r = 1
    for i in indices 
        if issorted(i)          
            for k in collect(permutations(i))
                ret[k...] = elements[r]
            end
            r += 1
        end
    end
    ret
end

mat = randn(6,10);
t = genstensor(Float64, 4,10)
bs = convert(BoxStructure{Float64}, t, 4);

function multimodemult{T <: AbstractFloat, N}(t::Array{T,N}, mat::Matrix{T})
    ret = t
    for i = 1:N
        ret = Tensors.modemult(ret, mat, i)
    end
    ret
end

multimodemult (generic function with 1 method)

In [5]:
s = size(bs)

N = 4
ret = NullableArray(Array{Float64, N}, fill(s[2], N)...)
for i = 1:s[2], j = 1:i, k = 1:j, l = 1:k
    ret[l,k,j,i] = (mm(i,j,k)[l].value)
end
ret[2,1,1,1];

LoadError: LoadError: UndefVarError: mm not defined
while loading In[5], in expression starting on line 5

In [6]:
function mm(i::Int,j::Int,k::Int)
    s = size(bs)
    ret = bs
    ret = mult(bs, m, [i], 4, s[2]);
    ret = mult(ret, m, [j, 1], 3, s[2])
    mult(ret, m, [k, 1, 1], 2, s[2])
end

N = 4
ret = NullableArray(Array{Float64, N}, fill(s[2], N)...)
for i = 1:s[2], j = 1:s[2], k = 1:s[2], l = 1:s[2]
    ret[i,l,k,j] = (mm(i,j,k)[l].value)
end
ret[1,1,1,2];

LoadError: LoadError: UndefVarError: mult not defined
while loading In[6], in expression starting on line 11

In [15]:
r = t
for i = 2:4
    r = Tensors.modemult(r, mat, i)
end

slise1(r, 3)[1,1,1,2];

In [7]:
function generateperm(i::Int, ar::Array{Int})
    ret = ar[:]
    ret[i], ret[1] = ar[1], ar[i]
    ret
end

function mult1{T <: AbstractFloat}(bs, m::NullableArray{Matrix{T}}, n::Int, s::Int, l::Int)
        N = 4
        temp = NullableArray(Array{T, N}, fill(s, (n-1))...)
        if n > 1
            for i in product(fill(1:s, (n-1))...)
            ll = [1,i...]
            temp[i...] = segmentmult1([generateperm(mode, ll)...], bs, m[l,:], n)
            end
        elseif n == 1
            temp = segmentmult1([1], bs, m[l,:], n)
        end
        temp
end

segmentmult3{T <: AbstractFloat, N}(k::Array{Int, 1}, m::NullableArray{Array{T, N}}, m1::NullableArray{Matrix{T}}, mode::Int = 1) =
mapreduce(j -> Tensors.modemult(m[generateperm(mode, [j, k[2:end]...])...].value, m1[k[1], j].value, mode), +, 1:size(m1, 1))

function mult2{T <: AbstractFloat}(bs, m::NullableArray{Matrix{T}}, n::Int, s::Int, l::Int)
        N = 4
        temp = NullableArray(Array{T, N}, fill(s, (n-1))...)
        if n > 1
            for i in product(fill(1:s, (n-1))...)
            ll = [i...,l]
            temp[i...] = segmentmult1([ll], bs, m, n)
            end
        elseif n == 1
            temp = segmentmult1([1], bs, m, n)
        end
        temp
end



mult2 (generic function with 1 method)

In [552]:
mode = 4
l = [1,1,2,1]
println(l)
s = segmentmult3([generateperm(mode, l)...], ts, m, mode)

[1,1,2,1]


2x2x2x2 Array{Float64,4}:
[:, :, 1, 1] =
  0.233089    0.00740024
 -0.0131956  -0.311787  

[:, :, 2, 1] =
  0.0589478  -0.137998
 -0.505058   -0.484187

[:, :, 1, 2] =
 -0.977339   0.640824
  1.06953   -2.37384 

[:, :, 2, 2] =
 0.59631   -1.41713 
 0.939613  -0.293411

In [2]:
i = [1,1,1,1]



4-element Array{Int64,1}:
 1
 1
 1
 1

In [695]:
ret = mult2(bs, m, 4, size(bs)[2], 1)
ret = mult2(ret, m, 3, size(bs)[2], 1)
ret = mult2(ret, m, 2, size(bs)[2], 1)


2-element NullableArrays.NullableArray{Array{Float64,4},1}:
 2x2x2x2 Array{Float64,4}:
[:, :, 1, 1] =
 0.118267  -0.649653
 0.857492   1.52028 

[:, :, 2, 1] =
 -0.649653  -1.54632
  1.52028    1.17321

[:, :, 1, 2] =
 -0.649653  -1.54632
  1.52028    1.17321

[:, :, 2, 2] =
 -1.54632  -3.57912
  1.17321  -1.71903
 2x2x2x2 Array{Float64,4}:
[:, :, 1, 1] =
  1.27678    0.296852
 -0.103145  -0.654222

[:, :, 2, 1] =
 0.979311   1.17319
 2.59747   -1.20095

[:, :, 1, 2] =
 0.979311   1.17319
 2.59747   -1.20095

[:, :, 2, 2] =
 4.38694   0.871882
 6.39352  -0.466234  